In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

def obtener_enlaces(url):
    # Realizar la petición HTTP a la página web
    respuesta = requests.get(url)
    # Si la petición fue exitosa, procesar el contenido
    if respuesta.status_code == 200:
        soup = BeautifulSoup(respuesta.content, 'html.parser')
        # Extraer todos los elementos <a> de la página
        enlaces = soup.find_all('a', href=True)
        # Crear una lista para almacenar los datos
        datos = []
        # Recorrer cada enlace y almacenar la información necesaria
        for enlace in enlaces:
            # Extraer el href del enlace
            href = enlace['href']
            # Extraer el texto del enlace, que es el nombre del archivo
            texto = enlace.text
            # Intentar extraer las observaciones que están en el contenido antes del enlace
            # Esto asume que el texto de observaciones está justo antes del enlace
            observaciones = enlace.previous_sibling
            if observaciones:
                observaciones = observaciones.strip()  #Remueve los espacios al principio y al final de la línea
                # Dividir las observaciones en partes
                partes = observaciones.split("   ")
                #print(partes)
                # Extraer la fecha y la hora (primeras dos partes)
                fecha_hora = ' '.join(partes[:1])
                #print(fecha_hora)
                # Extraer el tamaño (última parte)
                tamaño = partes[-1]
                #print(tamaño)
            else:
                fecha_hora = "No disponible"
                tamaño = "No disponible"
            # Añadir a la lista como una tupla
            datos.append((texto, "http://rtv-b2b-api-logs.vigiloo.net" + href, fecha_hora, tamaño))    # datos.append((texto, url + href, fecha_hora, tamaño))
        # Convertir la lista en un DataFrame
        df = pd.DataFrame(datos, columns=['NombreArchivo', 'URL', 'FechaHora', 'Tamaño'])
        df.drop(0,axis=0)
        # Convertir la columna 'FechaHora' a tipo datetime
        df['FechaHora'] = pd.to_datetime(df['FechaHora'], dayfirst=False, format='%m/%d/%Y %I:%M %p',errors='coerce') 
        # Convertir la columna 'Tamaño' a tipo numérico
        df['Tamaño'] = pd.to_numeric(df['Tamaño'], errors='coerce')
        df['Embarque'] = df['NombreArchivo'].str.extract(r'-(\d+)\.log')
        df.drop(0,axis=0, inplace=True)  # Borro la Primera línea que apunta al directorio padre        
        return df
    else:
        return "Error al acceder a la página"

In [2]:
# URL de la página de donde se desean extraer los enlaces
url = "http://rtv-b2b-api-logs.vigiloo.net/LoadUpload/"
archivos_df = obtener_enlaces(url)
if isinstance(archivos_df, pd.DataFrame):
   print(archivos_df.head(1))
else:
   print("Error")

            NombreArchivo                                                URL  \
1  638270843928515554.log  http://rtv-b2b-api-logs.vigiloo.net/LoadUpload...   

            FechaHora  Tamaño Embarque  
1 2023-08-08 08:39:00  2404.0      NaN  


In [3]:
# Convertir la columna 'FechaHora' a tipo datetime

import datetime

#salida['FechaHora'] = pd.to_datetime(salida['FechaHora'], dayfirst=False, format='%m/%d/%Y %I:%M %p') 
#salida['FechaHora'] = pd.to_datetime(salida['FechaHora'], format='%m/%d/%Y %I:%M %p')  # 10/16/2023  1:23 PM

#salida['FechaHora'] = pd.to_datetime(salida['FechaHora'], dayfirst=False).dt.strftime('%m/%d/%Y %I:%M %p')
archivos_df['FechaHora'] = pd.to_datetime(archivos_df['FechaHora'], dayfirst=False, format='%m/%d/%Y %I:%M %p',errors='coerce')   # Convierte en Fecha

In [15]:
print(datetime.datetime(year=2024,month=5,day=1))

2024-05-01 00:00:00


In [16]:
# Tomas las últimas 50 fílas válidas
# salida= archivos_df.tail(51)
# salida.drop(salida.index[-1], axis=0, inplace = True) # Elimina la última fila


salida = archivos_df[(archivos_df['FechaHora'] >=datetime.datetime(year=2024,month=5,day=1))]
# print(filtered_df)


## Explicación del Formato

%m: Representa el mes como un número con dos dígitos (01 a 12).

%d: Representa el día del mes como un número con dos dígitos (01 a 31).

%Y: Representa el año con cuatro dígitos.

%I: Representa la hora en formato de 12 horas (01 a 12).

%M: Representa los minutos.

%p: Representa AM o PM.

En nuestro caso como la fecha tiene los Dias y los Meses sin PADD hay que utilizar la siguiente opción

### pd.to_datetime(salida['FechaHora'], dayfirst=False).dt.strftime('%m/%d/%Y %I:%M %p')


https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

https://pandas.pydata.org/pandas-docs/version/1.0.1/reference/api/pandas.to_datetime.html

https://github.com/pandas-dev/pandas/issues/3341

In Linux "#" is replaced by "-":
%-d, %-H, %-I, %-j, %-m, %-M, %-S, %-U, %-w, %-W, %-y, %-Y
In Windows "-" is replaced by "#":
%#d, %#H, %#I, %#j, %#m, %#M, %#S, %#U, %#w, %#W, %#y, %#Y

#Linux
mydatetime.strftime('%-m/%d/%Y %-I:%M%p')
#Windows
mydatetime.strftime('%#m/%d/%Y %#I:%M%p')
Source: https://msdn.microsoft.com/en-us/library/fe06s4ak.aspx

In [6]:
salida.loc[salida.index[-1]]

NombreArchivo                     638511256466635778-834265169.log
URL              http://rtv-b2b-api-logs.vigiloo.net/LoadUpload...
FechaHora                                      2024-05-12 14:47:00
Tamaño                                                      1036.0
Embarque                                                 834265169
Name: 3661, dtype: object

In [4]:
salida.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46 entries, 3634 to 3679
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   NombreArchivo  46 non-null     object        
 1   URL            46 non-null     object        
 2   FechaHora      46 non-null     datetime64[ns]
 3   Tamaño         46 non-null     float64       
 4   Embarque       46 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 2.2+ KB


## Procesar un archivo JSON

* https://brightdata.es/blog/procedimientos/parse-json-data-with-python
* https://docs.python.org/3/tutorial/datastructures.html#dictionaries
* https://docs.python.org/3/tutorial/datastructures.html#more-on-lists


In [9]:
import urllib.request
import json
# Biscp un registro para probar

reg=2
url = salida.iloc[reg]['URL']

print(url)
print('Archivo : ',salida.iloc[reg]['NombreArchivo'])
print('Embarque: ',salida.iloc[reg]['Embarque'])
print('Fecha   : ',salida.iloc[reg]['FechaHora'])
print('Tamaño  : ',salida.iloc[reg]['Tamaño'])
embarque = pd.read_json(url)

http://rtv-b2b-api-logs.vigiloo.net/LoadUpload/638510162553663120-834264837.log
Archivo :  638510162553663120-834264837.log
Embarque:  834264837
Fecha   :  2024-05-11 08:24:00
Tamaño  :  907.0


### Ejemplo de Uso

https://stackoverflow.com/questions/74118717/create-new-dataframe-with-json-objects-from-another-dataframe#:~:text=Solution%20using%20pandas&text=It%20provides%20a%20DataFrame%20object,dataframe%20with%20the%20original%20one



In [ ]:
# EXPANDIR Embarque con el contenido de los registros de 2do nivel

# Defino la function que hace un parse al JSON string y retorna un dictionary
def parse_json(x):
  try:
    return json.dumps(x)
  except:
    return 'Error'

#Apply the function to the JSON column and store the result in a new column
embarque['json_dict'] = embarque['StopList'].apply(parse_json)

# Use the pandas.json_normalize function to convert the dictionary column into a new dataframe
df_json = pd.json_normalize(embarque['StopList'])

# Drop the original JSON column and merge the new dataframe
df = embarque.drop('StopList', axis=1)
embarque = pd.concat([embarque, df_json], axis=1)

In [18]:
embarque.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Id                 2 non-null      int64 
 1   Date               2 non-null      object
 2   LogisticGroupCode  2 non-null      object
 3   CarrierCode        2 non-null      int64 
 4   CarrierName        2 non-null      object
 5   StopList           2 non-null      object
 6   json_dict          2 non-null      object
 7   StopNumber         2 non-null      int64 
 8   OperationType      2 non-null      object
 9   DroppedNumber      2 non-null      int64 
 10  PickedNumber       2 non-null      int64 
 11  LocationCode       2 non-null      object
 12  LocationName       2 non-null      object
 13  LocationLatitude   0 non-null      object
 14  LocationLongitude  0 non-null      object
 15  Deliveries         1 non-null      object
dtypes: int64(5), object(11)
memory usage: 388.0+ byt

In [12]:
#embarque['json_dict'] = embarque['Deliveries'].apply(parse_json)

#df['json_dict'] = df['Deliveries'].apply(parse_json)

In [ ]:
# Ejemplo de datos (asegúrate de tener tus datos reales cargados)
# data = {...}
# df_deliveries = pd.DataFrame(data)

def process_deliveries(row):
    if row['DroppedNumber'] > 0 and pd.notna(row['Deliveries']):
        location = row['LocationName']
        # Convertir el contenido de la columna 'Deliveries' desde JSON a objeto Python si aún no está hecho
        if isinstance(row['Deliveries'], str):
            deliveries = eval(row['Deliveries'])
        else:
            deliveries = row['Deliveries']
        
        # Recorrer cada entrega en 'Deliveries'
        for delivery in deliveries:
            delivery_number = delivery['DeliveryNumber']            
            # Recorrer cada ítem en 'Items' de cada entrega
            for item in delivery['Items']:
                material_name = item['MaterialName']
                batch = item['Batch']
                qty = item['Qty']
                # Podemos elegir imprimir, almacenar o devolver esta información
                print(f" DeliveryNumber: {delivery_number}, LocationName: {location}, MaterialName: {material_name}, Batch: {batch}, Qty: {qty}")

# Aplicar la función a cada fila del DataFrame
#df.apply(process_deliveries, axis=1)
embarque.apply(process_deliveries, axis=1)


In [17]:
for i in range(len(salida)):
    if salida.iloc[i]['Embarque'] in ('834261368', '834261373', '834261712'):
        print('----------------------------------------------------------------')
        print('Embarque: ',salida.iloc[i]['Embarque'])
        url = salida.iloc[i]['URL']
        print('Archivo: ',salida.iloc[i]['FechaHora'],url)
        # Procesar cada Embarque
        embarque = pd.read_json(url)
        embarque['json_dict'] = embarque['StopList'].apply(parse_json)
        df_json = pd.json_normalize(embarque['StopList'])
        embarque = pd.concat([embarque, df_json], axis=1)
        print('Origen: ',embarque.iloc[0]['LogisticGroupCode'],' - Date: ',embarque.iloc[0]['Date'],' - Carrier: ',embarque.iloc[0]['CarrierName'])
        embarque.apply(process_deliveries, axis=1)

----------------------------------------------------------------
Embarque:  834261368
Archivo:  2024-05-09 15:57:00 http://rtv-b2b-api-logs.vigiloo.net/LoadUpload/638508706321030403-834261368.log
Origen:  SAPP08  - Date:  0001-01-01T00:00:00  - Carrier:  ZARCAM S A
 DeliveryNumber: 5011484115, LocationName: COMPAÑIA DE INSUMOS Y GRANOS S A, MaterialName: LA TIJERETA PLATINUM II,MON301512,48X20L, Batch: None, Qty: 19200.0
----------------------------------------------------------------
Embarque:  834261373
Archivo:  2024-05-09 15:57:00 http://rtv-b2b-api-logs.vigiloo.net/LoadUpload/638508706321030403-834261373.log
Origen:  SAPP08  - Date:  0001-01-01T00:00:00  - Carrier:  ZARCAM S A
 DeliveryNumber: 5011478200, LocationName: AGRO LEBEN SRL, MaterialName: RUP CONTROLMAX,AR,64x15KG BOX, Batch: None, Qty: 1905.0
 DeliveryNumber: 5011478200, LocationName: AGRO LEBEN SRL, MaterialName: HARNESS,AR,48x20L BT MON58415, Batch: None, Qty: 740.0
 DeliveryNumber: 5011478200, LocationName: AGRO LEBE

## Comparar con los Datos que están en SQL SERVER

In [34]:
import pyodbc

SERVER = '213.134.40.73,9595'
DATABASE = 'seamtrack'
USERNAME = 'eduardo.ettlin@vigiloo.com.ar'
PASSWORD = 'Aladelta10$'
OPTIONS= 'encrypt=no'

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD};{OPTIONS}'
print(connectionString)
conn = pyodbc.connect(connectionString) 

DRIVER={ODBC Driver 18 for SQL Server};SERVER=213.134.40.73,9595;DATABASE=seamtrack;UID=eduardo.ettlin@vigiloo.com.ar;PWD=Aladelta10$;encrypt=no


In [55]:
REGISTROS="""
'834261368',
'834261373',
'834261712'
"""


SQL_QUERY = f"""
SELECT   DISTINCT     
A.[RefId] as EMBARQUE
,D.RefId as DELIVERY
,A.[Date] as FECHA
,D.[PurchaseOrderIdRef] as PEDIDO
,D.[DestinationCode]
,D.[DestinationName]
,E.[MaterialCode]
,E.[MaterialName]
,E.[Batch]
,STR(E.[Qty],6) as QTY
,A.[LogisticGroupId]

FROM [seamtrack].[vigiloo].[Shipment] A
LEFT JOIN [seamtrack].[vigiloo].[ShipmentDestination] B
ON B.[ShipmentId] = A.Id
LEFT JOIN [seamtrack].[vigiloo].[Delivery] D
ON D.[ShipmentId] = A.Id
LEFT JOIN [seamtrack].[vigiloo].[DeliveryItem] E
ON E.[DeliveryId] = D.Id
    
WHERE A.[RefId] IN ( {REGISTROS}) ORDER BY A.RefId,D.RefId, D.[DestinationCode],E.[MaterialCode];
"""

In [56]:
print(SQL_QUERY)


SELECT   DISTINCT     
A.[RefId] as EMBARQUE
,D.RefId as DELIVERY
,A.[Date] as FECHA
,D.[PurchaseOrderIdRef] as PEDIDO
,D.[DestinationCode]
,D.[DestinationName]
,E.[MaterialCode]
,E.[MaterialName]
,E.[Batch]
,STR(E.[Qty],6) as QTY
,A.[LogisticGroupId]

FROM [seamtrack].[vigiloo].[Shipment] A
LEFT JOIN [seamtrack].[vigiloo].[ShipmentDestination] B
ON B.[ShipmentId] = A.Id
LEFT JOIN [seamtrack].[vigiloo].[Delivery] D
ON D.[ShipmentId] = A.Id
LEFT JOIN [seamtrack].[vigiloo].[DeliveryItem] E
ON E.[DeliveryId] = D.Id
    
WHERE A.[RefId] IN ( 
'834261368',
'834261373',
'834261712'
) ORDER BY A.RefId,D.RefId, D.[DestinationCode],E.[MaterialCode];



In [57]:
cursor = conn.cursor()
cursor.execute(SQL_QUERY)

In [58]:
records = cursor.fetchall()
for r in records:
    print(f"{r.EMBARQUE}\t{r.DELIVERY}\t{r.FECHA}\t{r.DestinationCode}\t{r.DestinationName}\t{r.MaterialName}\t{r.Batch}\t{r.QTY}\t{r.LogisticGroupId}")

834261368	5011484115	2024-05-10 00:00:00	0003391247	COMPAÑIA DE INSUMOS Y GRANOS S A	LA TIJERETA PLATINUM II,MON301512,48X20L	None	 19200	75247427-34FD-41A3-B7E0-F0B8312EADA8
834261373	5011478200	2024-05-10 00:00:00	0003107764	AGRO LEBEN SRL	HARNESS,AR,48x20L BT MON58415	None	   740	75247427-34FD-41A3-B7E0-F0B8312EADA8
834261373	5011478200	2024-05-10 00:00:00	0003107764	AGRO LEBEN SRL	RUP CONTROLMAX,AR,64x15KG BOX	None	  1980	75247427-34FD-41A3-B7E0-F0B8312EADA8
834261373	5011478200	2024-05-10 00:00:00	0003107764	AGRO LEBEN SRL	RUP TOP,AR,48x20L BT MON301515	None	  7000	75247427-34FD-41A3-B7E0-F0B8312EADA8
834261712	8480743658	2024-05-10 16:37:00	0004339120	ADECO AGROPECUARIA S.A.	PUCARA FS400 12X1L BOT AR	None	   400	F21EE1F9-4823-42B4-AD40-8C81F2847818


In [59]:
df = pd.DataFrame((tuple(t) for t in records)) 
df.columns = ['EMBARQUE','DELIVERY','FECHA','PEDIDO','DestinationCode','DestinationName','MaterialCode','MaterialName','Batch','QTY','LogisticGroupId']

In [64]:
# Adecuar el Grupo Logístico
df["LogisticGroupId"] = df["LogisticGroupId"].replace(
    { "75247427-34FD-41A3-B7E0-F0B8312EADA8": "Monsanto", "F21EE1F9-4823-42B4-AD40-8C81F2847818": "BAYER" })

In [65]:
df.head(10)

,EMBARQUE,DELIVERY,FECHA,PEDIDO,DestinationCode,DestinationName,MaterialCode,MaterialName,Batch,QTY,LogisticGroupId
0,834261368,5011484115,2024-05-10 00:00:00,0829025864,0003391247,COMPAÑIA DE INSUMOS Y GRANOS S A,30202284,"LA TIJERETA PLATINUM II,MON301512,48X20L",None,19200,Monsanto
1,834261373,5011478200,2024-05-10 00:00:00,0829020942,0003107764,AGRO LEBEN SRL,10434277,"HARNESS,AR,48x20L BT MON58415",None,740,Monsanto
2,834261373,5011478200,2024-05-10 00:00:00,0829020942,0003107764,AGRO LEBEN SRL,12331311,"RUP CONTROLMAX,AR,64x15KG BOX",None,1980,Monsanto
3,834261373,5011478200,2024-05-10 00:00:00,0829020942,0003107764,AGRO LEBEN SRL,30146700,"RUP TOP,AR,48x20L BT MON301515",None,7000,Monsanto
4,834261712,8480743658,2024-05-10 16:37:00,8450206770,0004339120,ADECO AGROPECUARIA S.A.,80897995,PUCARA FS400 12X1L BOT AR,None,400,BAYER


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   EMBARQUE         5 non-null      object        
 1   DELIVERY         5 non-null      object        
 2   FECHA            5 non-null      datetime64[ns]
 3   PEDIDO           5 non-null      object        
 4   DestinationCode  5 non-null      object        
 5   DestinationName  5 non-null      object        
 6   MaterialCode     5 non-null      object        
 7   MaterialName     5 non-null      object        
 8   Batch            0 non-null      object        
 9   QTY              5 non-null      object        
 10  LogisticGroupId  5 non-null      object        
dtypes: datetime64[ns](1), object(10)
memory usage: 572.0+ bytes
